In [8]:
import sys
sys.path.append('/home/comoz/main_project/playlist_project/')  # 현재 디렉토리를 Python 경로에 추가
import os
from argparse import ArgumentParser
from typing import List, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from mlp_baseline.datasets import MLPDataset
from mlp_baseline.models import MLPModel
from scipy.io import mmread
from torch.utils.data import DataLoader
from tqdm import tqdm
import random

train_matrix = mmread("/home/comoz/main_project/playlist_project/data/split_data/train_matrix.mtx").tocsr()
valid_matrix = mmread("/home/comoz/main_project/playlist_project/data/split_data/valid_matrix.mtx").tocsr()

neg_train_matrix = mmread("/home/comoz/main_project/playlist_project/data/negative_sample/default/train_default_5_epochs_4_negatives.mtx").tocsr()
neg_valid_matrix = mmread("/home/comoz/main_project/playlist_project/data/negative_sample/default/valid_default_5_epochs_4_negatives.mtx").tocsr()

playlist_matrix = mmread("/home/comoz/main_project/playlist_project/data/playlist_song_matrix_50_50.mtx").tocsr()
# 데이터셋 생성
train_dataset = MLPDataset(train_matrix, neg_train_matrix, num_negatives=4)
valid_dataset = MLPDataset(valid_matrix, neg_valid_matrix, num_negatives=4)

In [9]:
import csv
song_names = {}
with open("/home/comoz/main_project/playlist_project/data/song_info_50_50.csv") as f:
    reader = csv.reader(f)
    next(reader)
    for song_id, song_name, artist_name in reader:
        song_names[int(song_id)] = (song_name, artist_name)


In [10]:
# 네거티브 샘플링 & 데이터 셋 테스트

user_id, pos_song_ids, neg_song_ids = train_dataset[random.randint(0, len(train_dataset))]

print(f"출력 테스트")
print(f"유저 아이디: {user_id}")
print(f"양성 샘플 노래 아이디: {pos_song_ids}")
print(f"음성 샘플 노래 아이디: {neg_song_ids}")

for pos_song_id in pos_song_ids:
    pos_song_id = int(pos_song_id)
    print(song_names[pos_song_id])

print("\n네거티브 샘플 노래 확인:")
for neg_song_id in neg_song_ids:
    neg_song_id = int(neg_song_id)  
    print(song_names[neg_song_id])

# playlist_matrix에서 해당 유저의 노래 리스트 확인
user_songs_from_matrix = playlist_matrix[user_id,:]


# pos_song_id가 해당 유저의 노래인지 확인
print("\n해당 유저의 노래가 맞는지 확인:")

print(f"positive_song_ids: {pos_song_ids}")
for pos_song_id in pos_song_ids:
    pos_song_id = int(pos_song_id)
    # train_matrix에서 해당 유저-노래 쌍의 값 확인
    interaction_value = playlist_matrix[user_id, pos_song_id]
    if interaction_value > 0:
        print(f"{song_names[pos_song_id]}: 유저의 노래가 맞습니다")
    else:
        print(f"{song_names[pos_song_id]}:  유저의 노래가 아닙니다!")
    print("--------------------------------\n")

print(f"negative_song_ids: {neg_song_ids}")
for neg_song_id in neg_song_ids:
    neg_song_id = int(neg_song_id)
    print(f"neg_song_id: {neg_song_id}")
    interaction_value = playlist_matrix[user_id, neg_song_id]
    if interaction_value > 0:
        print(f"{song_names[neg_song_id]}: 유저의 노래가 맞습니다!")
    else:
        print(f"{song_names[neg_song_id]}: 유저의 노래가 아닙니다!")
    print("--------------------------------")




출력 테스트
유저 아이디: tensor([7289])
양성 샘플 노래 아이디: tensor([16400])
음성 샘플 노래 아이디: tensor([1095, 6537, 7123, 4547])
('Running On Empty', 'Jackson Browne')

네거티브 샘플 노래 확인:
('Kings and Queens', 'Thirty Seconds To Mars')
("I'm So Sick", 'Flyleaf')
('Song For The Dead', 'Queens Of The Stone Age')
("Why Don't We Go There", 'One Direction')

해당 유저의 노래가 맞는지 확인:
positive_song_ids: tensor([16400])
('Running On Empty', 'Jackson Browne'): 유저의 노래가 맞습니다
--------------------------------

negative_song_ids: tensor([1095, 6537, 7123, 4547])
neg_song_id: 1095
('Kings and Queens', 'Thirty Seconds To Mars'): 유저의 노래가 아닙니다!
--------------------------------
neg_song_id: 6537
("I'm So Sick", 'Flyleaf'): 유저의 노래가 아닙니다!
--------------------------------
neg_song_id: 7123
('Song For The Dead', 'Queens Of The Stone Age'): 유저의 노래가 아닙니다!
--------------------------------
neg_song_id: 4547
("Why Don't We Go There", 'One Direction'): 유저의 노래가 아닙니다!
--------------------------------


In [118]:
print(f"playlist_matrix shape: {playlist_matrix.shape}")

raw_items = set(playlist_matrix[0,:].nonzero()[1])
print(f"raw_items: {raw_items}")


playlist_matrix shape: (10493, 27045)
raw_items: {np.int32(0), np.int32(1), np.int32(2), np.int32(3), np.int32(4), np.int32(5), np.int32(6), np.int32(7), np.int32(8), np.int32(9), np.int32(10), np.int32(11), np.int32(12), np.int32(13), np.int32(14), np.int32(15), np.int32(16), np.int32(17), np.int32(18), np.int32(19), np.int32(20), np.int32(21), np.int32(22), np.int32(23), np.int32(24), np.int32(25), np.int32(26), np.int32(27), np.int32(28), np.int32(29), np.int32(30), np.int32(31), np.int32(32), np.int32(33), np.int32(34), np.int32(35), np.int32(36), np.int32(37), np.int32(38), np.int32(39), np.int32(40), np.int32(41), np.int32(42), np.int32(43), np.int32(44), np.int32(45), np.int32(46), np.int32(47), np.int32(48), np.int32(49), np.int32(50), np.int32(51)}


In [11]:
from collections import Counter
from tqdm import tqdm

song_counter = Counter()
for user_id, pos_song_ids, neg_song_ids in tqdm(train_dataset):
    song_counter.update(pos_song_ids.tolist())
    

100%|██████████| 2281558/2281558 [00:21<00:00, 107692.69it/s]


In [12]:

most_common_song = song_counter.most_common(20)
for song_id, count in most_common_song:
    print(f"노래 아이디: {song_id}, 노래 등장 횟수: {count}")
    print(f"노래 이름: {song_names[int(song_id)]}")
    print("--------------------------------")

노래 아이디: 284, 노래 등장 횟수: 672
노래 이름: ('Electric Feel', 'MGMT')
--------------------------------
노래 아이디: 2815, 노래 등장 횟수: 672
노래 이름: ('Losing My Religion', 'R.E.M.')
--------------------------------
노래 아이디: 8076, 노래 등장 횟수: 667
노래 이름: ('I Need Your Love', 'Calvin Harris')
--------------------------------
노래 아이디: 683, 노래 등장 횟수: 665
노래 이름: ('On Top Of The World', 'Imagine Dragons')
--------------------------------
노래 아이디: 2339, 노래 등장 횟수: 659
노래 이름: ('Hey Ya! - Radio Mix / Club Mix', 'OutKast')
--------------------------------
노래 아이디: 333, 노래 등장 횟수: 658
노래 이름: ('Take a Walk', 'Passion Pit')
--------------------------------
노래 아이디: 7202, 노래 등장 횟수: 658
노래 이름: ('Firework', 'Katy Perry')
--------------------------------
노래 아이디: 3784, 노래 등장 횟수: 657
노래 이름: ('Team', 'Lorde')
--------------------------------
노래 아이디: 6881, 노래 등장 횟수: 657
노래 이름: ('Levels - Radio Edit', 'Avicii')
--------------------------------
노래 아이디: 1594, 노래 등장 횟수: 655
노래 이름: ('Breezeblocks', 'alt-J')
--------------------------------
노

In [15]:
# song_info에서 "포함된 노래 찾기
quotes_songs = []
for song_name, artist_name in song_names.values():
    if '"' in song_name or '"' in artist_name or "'" in song_name or "'" in artist_name:
        quotes_songs.append((song_name, artist_name))

print(f"따옴표가 포함된 노래 목록: {len(quotes_songs)}")
print("--------------------------------")
for song_name, artist_name in quotes_songs:
    print(f"노래 제목: {song_name}")
    print(f"아티스트 이름: {artist_name}")
    print("--------------------------------")


따옴표가 포함된 노래 목록: 2578
--------------------------------
노래 제목: (What's So Funny 'Bout) Peace, Love And Understanding
아티스트 이름: Elvis Costello & The Attractions
--------------------------------
노래 제목: Don't Dream It's Over
아티스트 이름: Crowded House
--------------------------------
노래 제목: I'd Rather Be With You [Radio Edit]
아티스트 이름: Joshua Radin
--------------------------------
노래 제목: Little Lovin'
아티스트 이름: Lissie
--------------------------------
노래 제목: Oliver's Army
아티스트 이름: Elvis Costello & The Attractions
--------------------------------
노래 제목: AKA... What A Life!
아티스트 이름: Noel Gallagher's High Flying Birds
--------------------------------
노래 제목: Free Fallin'
아티스트 이름: Tom Petty
--------------------------------
노래 제목: L'Estasi Dell'oro (The Ecstasy Of Gold) - 2004 Digital Remaster
아티스트 이름: Ennio Morricone
--------------------------------
노래 제목: Serenade No. 13 in G Major, K. 525 Eine Kleine Nachtmusik": I: Allegro"
아티스트 이름: Wolfgang Amadeus Mozart
--------------------------------
노래 제목: Burn

In [2]:
import pandas as pd

# 데이터 로드 (CSV 파일 경로는 실제 경로로 수정해주세요)
df = pd.read_csv(f"/home/comoz/main_project/playlist_project/data/spotify_dataset.csv", skipinitialspace=True, on_bad_lines="skip") 

# 1. user_id 유니크 카운트
unique_users = df['user_id'].nunique()
print(f"유니크 유저 수: {unique_users}")

# 2. user_id와 playlistname 조합의 유니크 카운트
# user_id와 playlistname을 결합
df['user_playlist'] = df['user_id'].astype(str) + "_" + df['playlistname']
unique_user_playlists = df['user_playlist'].nunique()
print(f"\nuser_id와 playlist 조합의 유니크 수: {unique_user_playlists}")

# 상위 10개 가장 많이 등장하는 user_playlist 조합 확인
top_user_playlists = df['user_playlist'].value_counts().head(10)
print("\n가장 많이 등장하는 user_playlist 조합 TOP 10:")
print(top_user_playlists)

# user_id별 플레이리스트 개수
playlists_per_user = df.groupby('user_id')['playlistname'].nunique()
print("\n유저별 플레이리스트 수 통계:")
print(playlists_per_user.describe())

ParserError: Error tokenizing data. C error: Expected 4 fields in line 14735, saw 5
